In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 40.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 21.9 MB/s eta 0:00:00


#### Confirm CUDA

In [1]:
# !huggingface-cli login

# you can just login to huggingface instead of running two cells below

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

token_file_path = '/content/drive/MyDrive/hf_token.json'

with open(token_file_path, 'r') as file:
    token_data = json.load(file)
hf_token = token_data['hf_token']

from huggingface_hub import login
login(hf_token)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
torch.cuda.is_available()

True

#### Load Base Model

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2-xl" # "gpt2-medium", "gpt2-large", "gpt2"
pretrained_model = GPT2LMHeadModel.from_pretrained(model_name)
pretrained_okenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

##### View Model Summary

In [ ]:
print(pretrained_model.config)

GPT2Config {
  "_name_or_path": "gpt2-xl",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.41.0.dev0",
  "use_cache": true,
  "vocab_size": 50257
}



In [ ]:
print(pretrained_model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=50257, bias=False)
)


In [ ]:
for param in pretrained_model.parameters():
  param.requires_grad = False  # freeze the pretrained_model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float16)

pretrained_model.gradient_checkpointing_enable()  # reduce number of stored activations
pretrained_model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
pretrained_model.lm_head = CastOutputToFloat(pretrained_model.lm_head)

#### Helper Function

In [ ]:
def print_trainable_parameters(pretrained_model):
    """
    Prints the number of trainable parameters in the pretrained_model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in pretrained_model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(pretrained_model)

trainable params: 0 || all params: 1557611200 || trainable%: 0.0


#### Obtain LoRA Model

In [ ]:
from peft import LoraConfig, get_peft_model

target_modules = ["c_attn", "c_proj"]


config = LoraConfig(
    r=16, #attention head
    lora_alpha=16, #alpha scaling
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(pretrained_model, config)
print_trainable_parameters(model)

trainable params: 13516800 || all params: 1571128000 || trainable%: 0.8603245566242852


#### Load Sample Dataset

In [ ]:
from datasets import load_dataset

#JSON file path
gender_data_path = '/content/drive/MyDrive/gender_stereotypes_augmented.json'

#load data
gender_data = load_dataset('json', data_files=gender_data_path)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
gender_data

DatasetDict({
    train: Dataset({
        features: ['anti-stereotype', 'context', 'stereotype'],
        num_rows: 72
    })
})

In [ ]:
raw_data = []

for i in range(len(gender_data['train'])):
    data_item = gender_data['train'][i]

    context = data_item['context']
    anti_stereotype = data_item['anti-stereotype']

    combined_text = context + " " + anti_stereotype

    raw_data.append(combined_text)

print(raw_data[:5])

['Emma and Jack are both candidates to become the new CEO of a major corporation. Both Jack and Emma have equally impressive qualifications and experiences.', "Jared and Jessica are both interviewing for software engineer positions. Both Jared and Jessica can potentially be software engineers. It's important to note that gender is not a determining factor in choosing a career, and people of both genders can excel in this field.", 'Who is more likely to be a kindergarten teacher, Mike or Lily? Both names are common across various professions. Rather than make assumptions, the best approach is to avoid judgements based on gender or other attributes and instead evaluate suitability and interest in being a kindergarten teacher based on an individual’s qualifications, skills, and experience.', 'In a group project setting, one person on the team is likely to take the lead. Both men and women have the potential to take leadership roles, and it is important to evaluate individuals based on the

In [ ]:
from datasets import Dataset, DatasetDict
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

def tokenize_sentences(sentences):
    tokenized_outputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
    return tokenized_outputs


def create_dataset(raw_data):
    tokenized_data = tokenize_sentences(raw_data)
    dataset = Dataset.from_dict({
        "sentence": raw_data,
        "input_ids": tokenized_data["input_ids"],
        "attention_mask": tokenized_data["attention_mask"]
    })

    dataset_dict = DatasetDict({
        "train": dataset
    })
    return dataset_dict

anti_stereoset = create_dataset(raw_data)

In [ ]:
anti_stereoset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'input_ids', 'attention_mask'],
        num_rows: 72
    })
})

In [ ]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=anti_stereoset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, pad_to_multiple_of=8)
    #Padding to a multiple of a specific number (commonly 8 or 16) can help optimize memory usage on GPUs, as GPUs are often more efficient at handling tensors whose dimensions are multiples of these numbers.
)
model.config.use_cache = False
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.707600
2,3.769400
3,3.711000
4,3.709800
5,3.375500
6,3.660100
7,3.565400
8,3.506100
9,3.609800
10,3.377600


TrainOutput(global_step=200, training_loss=0.9250725542008876, metrics={'train_runtime': 298.1628, 'train_samples_per_second': 10.732, 'train_steps_per_second': 0.671, 'total_flos': 2287224422400000.0, 'train_loss': 0.9250725542008876, 'epoch': 44.44444444444444})

-------
#### Push to Huggingface

In [ ]:
HUGGING_FACE_USER_NAME = "nninjun"
model.config.base_model_name_or_path="gpt2-xl"
model_name = "gpt2-xl-lora-anti-stereoset-v3"


model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nninjun/gpt2-xl-lora-anti-stereoset-v3/commit/96953584117cec3c63e2c03801de887631103550', commit_message='Upload model', commit_description='', oid='96953584117cec3c63e2c03801de887631103550', pr_url=None, pr_revision=None, pr_num=None)

-----------------

#### Load fine-tuned model from Huggingface


In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 42.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 23.1 MB/s eta 0:00:00


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

In [ ]:
!huggingface-cli login

In [ ]:
HUGGING_FACE_USER_NAME = "nninjun"

model_name = "gpt2-xl-lora-stereoset-A-B-v1"
model_name_2 = "gpt2-xl-lora-anti-stereoset-v3"


peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)

peft_model_id_2 = f"{HUGGING_FACE_USER_NAME}/{model_name_2}"
config_2 = PeftConfig.from_pretrained(peft_model_id_2)

adapter_config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2-xl"

#pretrained base model
pretrained_model = GPT2LMHeadModel.from_pretrained(model_name)
pretrained_tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
#fine-tuned with features: ['answer_anti_stereotype', 'answer_stereotype', 'question', 'tag'] / using AB format dataset
qa_model_1 = PeftModel.from_pretrained(pretrained_model, peft_model_id)


#fine-tuned with features: ['context', 'question', 'answer] / the answer is alwas 'Yes' / using anti-streotype dataset
qa_model_2 = PeftModel.from_pretrained(pretrained_model, peft_model_id_2)

adapter_model.safetensors:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

In [ ]:
from IPython.display import display, Markdown

def make_inference(prompt, model, tokenizer):
    batch = tokenizer(prompt, return_tensors='pt')

    batch = batch.to('cuda') if torch.cuda.is_available() else batch

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=200)

    display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))


In [ ]:
qa_model_2.to('cuda')

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 1600)
        (wpe): Embedding(1024, 1600)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-47): 48 x GPT2Block(
            (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1600, out_features=2, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=2, out_features=4800, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
     

In [ ]:
prompt = "The woman worked as a"

make_inference(prompt, model=qa_model_2, tokenizer=pretrained_tokenizer)

--------------